In [1]:
import boto3

sqs = boto3.client('sqs')

In [ ]:
response = sqs.get_queue_url(QueueName='highlighter.fifo')
queue_url = response["QueueUrl"]

In [29]:
response = sqs.send_message(
    QueueUrl=queue_url,
    MessageAttributes={
        "Title": {
            "DataType": "String",
            "StringValue": "The Whistler"
        },
        "Author": {
            "DataType": "String",
            "StringValue": "John Grisham"
        },
        "WeeksOn": {
            "DataType": "Number",
            "StringValue": "6"
        }
    },
    MessageBody=(
        "Information about current NY Times fiction bestseller for "
        "week of 12/11/2016."
    ),
    MessageGroupId="1",
    MessageDeduplicationId="1",
)

In [ ]:
print(response["MessageId"])

In [54]:
response = sqs.receive_message(
    QueueUrl=queue_url,
    AttributeNames=[
        'All'
    ],
    MaxNumberOfMessages=1,
    MessageAttributeNames=[
        'All'
    ],
    VisibilityTimeout=100,
    WaitTimeSeconds=0
)

In [ ]:
message = response['Messages'][0]
receipt_handle = message['ReceiptHandle']

# Delete received message from queue
sqs.delete_message(
    QueueUrl=queue_url,
    ReceiptHandle=receipt_handle
)
print('Received and deleted message: %s' % message)

In [58]:
sqs.purge_queue(QueueUrl=queue_url)

{'ResponseMetadata': {'RequestId': '9e8fc2bc-b1ae-5b4c-ac72-9c120990bc99',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9e8fc2bc-b1ae-5b4c-ac72-9c120990bc99',
   'date': 'Fri, 03 May 2024 10:01:34 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [41]:
import base64
a = base64.b64decode(message["Body"].encode())

In [70]:
with open("vid.ts", "wb") as video_file:
    while True:
        response = sqs.receive_message(
            QueueUrl=queue_url,
            AttributeNames=[
                'All'
            ],
            MaxNumberOfMessages=1,
            MessageAttributeNames=[
                'All'
            ],
            VisibilityTimeout=100,
            WaitTimeSeconds=0
        )
        try:
            message = response['Messages'][0]
            receipt_handle = message['ReceiptHandle']

            # Delete received message from queue
            sqs.delete_message(
                QueueUrl=queue_url,
                ReceiptHandle=receipt_handle
            )
            a = base64.b64decode(message["Body"].encode())

            video_file.write(a)
        except:
            break